# Model serialization

## Import model

In [ ]:
from __future__ import annotations

import json

import sympy as sp
from ampform.dynamics import BlattWeisskopfSquared
from ampform.dynamics.phasespace import BreakupMomentumSquared
from ampform.kinematics.phasespace import Kallen
from IPython.display import JSON, Markdown, Math, display

from ampform_dpd import DefinedExpression
from ampform_dpd.dynamics import (
    BreitWigner,
    BuggBreitWigner,
    ChannelArguments,
    EnergyDependentWidth,
    FormFactor,
    MultichannelBreitWigner,
    P,
    SimpleBreitWigner,
)
from ampform_dpd.io import as_markdown_table, aslatex
from ampform_dpd.io.serialization.decay import (
    _get_decay_chains,
    get_final_state,
    to_decay,
)
from ampform_dpd.io.serialization.dynamics import (
    formulate_breit_wigner,
    formulate_dynamics,
    formulate_form_factor,
    formulate_multichannel_breit_wigner,
    to_mandelstam_symbol,
    to_mass_symbol,
)
from ampform_dpd.io.serialization.format import (
    ModelDefinition,
    Propagator,
    get_function_definition,
)

In [ ]:
with open("Lc2ppiK.json") as stream:
    MODEL_DEFINITION = json.load(stream)

In [ ]:
JSON(MODEL_DEFINITION)

## Construct `ThreeBodyDecay`

In [ ]:
def to_latex(name: str) -> str:
    latex = {
        "Lc": R"\Lambda_c^+",
        "pi": R"\pi^+",
        "K": "K^-",
        "p": "p",
    }.get(name)
    if latex is not None:
        return latex
    mass_str = name[1:].strip("(").strip(")")
    subsystem_letter = name[0]
    subsystem = {"D": "D", "K": "K", "L": R"\Lambda"}.get(subsystem_letter)
    if subsystem is None:
        return name
    return f"{subsystem}({mass_str})"

In [ ]:
DECAY = to_decay(MODEL_DEFINITION, to_latex=to_latex)
Math(aslatex(DECAY))

In [ ]:
Markdown(as_markdown_table(DECAY))

## Dynamics

:::{seealso} [RUB-EP1/amplitude-serialization#22](https://github.com/RUB-EP1/amplitude-serialization/issues/22)
:::

In [ ]:
CHAIN_DEFS = _get_decay_chains(MODEL_DEFINITION)

### Vertices

#### Blatt-Weisskopf form factor

In [ ]:
z = sp.Symbol("z", nonnegative=True)
s, m1, m2, L, d = sp.symbols("s m1 m2 L R")
exprs = [
    FormFactor(s, m1, m2, L, d),
    BlattWeisskopfSquared(z, L),
    BreakupMomentumSquared(s, m1, m2),
]
Math(aslatex({e: e.doit(deep=False) for e in exprs}))

In [ ]:
ff_L1520 = formulate_form_factor(
    vertex=CHAIN_DEFS[2]["vertices"][0],
    model=MODEL_DEFINITION,
)
Math(aslatex(ff_L1520))

### Propagators

#### Breit-Wigner

In [ ]:
x, y, z = sp.symbols("x:z")
s, m0, Γ0, m1, m2, L, d = sp.symbols("s m0 Gamma0 m1 m2 L R")
exprs = [
    BreitWigner(s, m0, Γ0, m1, m2, L, d),
    SimpleBreitWigner(s, m0, Γ0),
    EnergyDependentWidth(s, m0, Γ0, m1, m2, L, d),
    FormFactor(s, m1, m2, L, d),
    P(s, m1, m2),
    Kallen(x, y, z),
]
Math(aslatex({e: e.doit(deep=False) for e in exprs}))

In [ ]:
K892_BW = formulate_breit_wigner(
    propagator=CHAIN_DEFS[20]["propagators"][0],
    resonance=to_latex(CHAIN_DEFS[20]["name"]),
    model=MODEL_DEFINITION,
)
Math(aslatex(K892_BW))

#### Multi-channel Breit-Wigner

In [ ]:
x, y, z = sp.symbols("x:z")
s, m0, Γ0, m1, m2, L, d = sp.symbols("s m0 Gamma0 m1 m2 L R")
channels = [
    ChannelArguments(
        sp.Symbol(f"Gamma{i}"),
        sp.Symbol(f"m_{{a,{i}}}"),
        sp.Symbol(f"m_{{b,{i}}}"),
        sp.Symbol(f"L{i}"),
        d,
    )
    for i in [1, 2]
]
exprs = [
    MultichannelBreitWigner(s, m0, channels),
    BreitWigner(s, m0, Γ0, m1, m2, L, d),
    BreitWigner(s, m0, Γ0),
    EnergyDependentWidth(s, m0, Γ0, m1, m2, L, d),
    FormFactor(s, m1, m2, L, d),
    P(s, m1, m2),
    Kallen(x, y, z),
]
Math(aslatex({e: e.doit(deep=False) for e in exprs}))

In [ ]:
L1405_Flatte = formulate_multichannel_breit_wigner(
    propagator=CHAIN_DEFS[0]["propagators"][0],
    resonance=to_latex(CHAIN_DEFS[0]["name"]),
    model=MODEL_DEFINITION,
)
Math(aslatex(L1405_Flatte))

#### Breit-Wigner with exponential

The model contains one lineshape function that is not standard, so we have to implement a custom propagator dynamics builder for this.

In [ ]:
s, m0, Γ0, m1, m2, γ = sp.symbols("s m0 Gamma0 m1 m2 gamma")
expr = BuggBreitWigner(s, m0, Γ0, m1, m2, γ)
Math(aslatex({expr: expr.doit(deep=False)}))

In [ ]:
CHAIN_DEFS[18]

In [ ]:
get_function_definition("K700_BuggBW", MODEL_DEFINITION)

In [ ]:
def formulate_bugg_breit_wigner(
    propagator: Propagator, resonance: str, model: ModelDefinition
) -> DefinedExpression:
    function_definition = get_function_definition(propagator["parametrization"], model)
    node = propagator["node"]
    i, j = node
    s = to_mandelstam_symbol(node)
    mass = sp.Symbol(f"m_{{{resonance}}}", nonnegative=True)
    width = sp.Symbol(Rf"\Gamma_{{{resonance}}}", nonnegative=True)
    γ = sp.Symbol(Rf"\gamma_{{{resonance}}}")
    m1 = to_mass_symbol(i)
    m2 = to_mass_symbol(j)
    final_state = get_final_state(model)
    return DefinedExpression(
        expression=BuggBreitWigner(s, mass, width, m1, m2, γ),
        definitions={
            mass: function_definition["mass"],
            width: function_definition["width"],
            m1: final_state[i].mass,
            m2: final_state[j].mass,
            γ: function_definition["slope"],
        },
    )

In [ ]:
CHAIN_18 = CHAIN_DEFS[18]
K700_BuggBW = formulate_bugg_breit_wigner(
    propagator=CHAIN_18["propagators"][0],
    resonance=to_latex(CHAIN_18["name"]),
    model=MODEL_DEFINITION,
)
Math(aslatex(K700_BuggBW))

#### General propagator dynamics builder

In [ ]:
DYNAMICS_BUILDERS = {
    "BreitWignerWidthExpLikeBugg": formulate_bugg_breit_wigner,
}

In [ ]:
exprs = [
    formulate_dynamics(CHAIN_DEFS[0], MODEL_DEFINITION, to_latex, DYNAMICS_BUILDERS),
    formulate_dynamics(CHAIN_DEFS[18], MODEL_DEFINITION, to_latex, DYNAMICS_BUILDERS),
    formulate_dynamics(CHAIN_DEFS[20], MODEL_DEFINITION, to_latex, DYNAMICS_BUILDERS),
]
for expr in exprs:
    display(Math(aslatex(expr)))